#### Imports

In [56]:
import pandas as pd 
import os

In [57]:
users_prep_df=pd.DataFrame(pd.read_csv('02Prep/usersPrep.csv'))
trainings_prep_df=pd.DataFrame(pd.read_csv('02Prep/trainingsPrep.csv'))
responses_prep_df=pd.DataFrame(pd.read_csv('02Prep/responsesPrep.csv'))
organisation_review_prep_df=pd.DataFrame(pd.read_csv('02Prep/organisationReviewPrep.csv'))

In [58]:
os.makedirs('03Transform',exist_ok=True)

##### 1st KPI (AVG SCORES IN EACH TRAINING && SUCCESS RATE OF EACH TRAINING)

In [59]:
# trainings_prep_df
avg_scores=responses_prep_df.groupby('trainingId')['score'].mean().reset_index().round(2)
avg_scores.columns=['trainingId','avg_scores']
print(avg_scores.sort_values(by='avg_scores',ascending=False))


    trainingId  avg_scores
16          17       57.02
83          84       56.52
38          39       55.66
9           10       55.48
92          93       55.07
..         ...         ...
36          37       45.06
81          82       44.72
79          80       44.65
33          34       43.61
10          11       40.88

[100 rows x 2 columns]


In [60]:
success_rate = responses_prep_df.groupby('trainingId').apply(lambda x: (x['score'] >= 60).mean() * 100).reset_index().round(2)
success_rate.columns = ['trainingId', 'success_rate_percentage']
print(success_rate.sort_values(by='success_rate_percentage',ascending=False))

    trainingId  success_rate_percentage
38          39                    52.04
98          99                    50.60
65          66                    50.56
62          63                    50.54
24          25                    50.52
..         ...                      ...
17          18                    33.00
99         100                    32.41
33          34                    31.82
79          80                    29.20
10          11                    24.51

[100 rows x 2 columns]


C:\Users\dell\AppData\Local\Temp\ipykernel_15392\2345469540.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  success_rate = responses_prep_df.groupby('trainingId').apply(lambda x: (x['score'] >= 60).mean() * 100).reset_index().round(2)


In [61]:
avg_success_rate_df=pd.merge(avg_scores, success_rate, on='trainingId')
avg_success_rate_df.to_csv('03Transform/trainingAvgSuccessRateTrans.csv',index=False)

##### 2nd KPI (RESPONSES OF EACH TRAININGS WITH HIGHEST SCORERS OF EACH TRAININGS)

In [62]:
response_count=responses_prep_df.groupby('trainingId').size().reset_index(name='total_responses')
print(response_count)


    trainingId  total_responses
0            1               91
1            2              110
2            3               76
3            4               99
4            5              103
..         ...              ...
95          96              118
96          97               97
97          98               87
98          99               83
99         100              108

[100 rows x 2 columns]


In [63]:
max_scores = responses_prep_df.loc[responses_prep_df.groupby('trainingId')['score'].idxmax()][['trainingId','userId','score']]
max_scores.columns = ['trainingId', 'userId_with_highest_score', 'highest_score']
print(max_scores)


      trainingId  userId_with_highest_score  highest_score
4777           1                        172            100
176            2                        257            100
3564           3                         32             99
2479           4                        287             99
9147           5                        221            100
...          ...                        ...            ...
427           96                        205            100
403           97                         19            100
3994          98                        174            100
2790          99                        139            100
1598         100                        243            100

[100 rows x 3 columns]


In [64]:
(pd.merge(response_count,max_scores, on='trainingId')).to_csv('03Transform/trainingOverviewsTrans.csv')